In [1]:
!pip install datasets
#!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
dataset = load_dataset("dair-ai/emotion")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
from datasets import load_dataset
dataset = load_dataset("dair-ai/emotion")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [6]:
dataset.data

{'train': MemoryMappedTable
 text: string
 label: int64
 ----
 text: [["i didnt feel humiliated","i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake","im grabbing a minute to post i feel greedy wrong","i am ever feeling nostalgic about the fireplace i will know that it is still on the property","i am feeling grouchy",...,"i should have been depressed but i was actually feeling inspired","i feel like not enough people my age actually think that most are pretty devastated that their s have come and gone","i get home i laze around in my pajamas feeling grouchy","i am feeling pretty homesick this weekend","i started out feeling really optimistic and driven for this paper coz it was gonna teach me the meaning and ways of being a leader"],["i need to do the best i possibly can do and even when i get out at i feel too listless to study like right now","i drove us to the car parts place and terry feels like im safe to drive again so yip

In [7]:
pd.Series(dataset['train'][:]['label']).value_counts()

1    5362
0    4666
3    2159
4    1937
2    1304
5     572
dtype: int64

In [8]:
dataset['train'][0:4]
X_train = dataset['train'][:]['text'].copy()
y_train = dataset['train'][:]['label'].copy()
y_train[0:4]
X_val = dataset['validation'][:]['text'].copy()
y_val = dataset['validation'][:]['label'].copy()
X_test = dataset['test'][:]['text'].copy()
y_test = dataset['test'][:]['label'].copy()

In [9]:
X_all = np.concatenate((X_train, X_val, X_test), axis=0)
X_all.shape

(20000,)

In [10]:
X_all

array(['i didnt feel humiliated',
       'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
       'im grabbing a minute to post i feel greedy wrong', ...,
       'i feel that i am useful to my people and that gives me a great feeling of achievement',
       'im feeling more comfortable with derby i feel as though i can start to step out my shell',
       'i feel all weird when i have to meet w people i text but like dont talk face to face w'],
      dtype='<U300')

In [11]:
y_all = np.concatenate((y_train, y_val, y_test), axis = 0)
y_all.shape

(20000,)

In [12]:
y_all

array([0, 0, 3, ..., 1, 1, 4])

In [13]:
unique_values, counts = np.unique(y_all, return_counts=True)
value_counts = dict(zip(unique_values, counts))
print("Value Counts:")
for value, count in value_counts.items():
    print(f"{value}: {count}")

Value Counts:
0: 5797
1: 6761
2: 1641
3: 2709
4: 2373
5: 719


In [14]:
from datasets import load_dataset
from transformers import pipeline
import pandas as pd
import plotly.express as px
import torch

In [15]:
device = torch.device("cuda")

# A more old-school method

## BertTokenizer

In [17]:
# an short example of how the hell the bertTokenizer works.

'''
resource: https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f
padding : to pad each sequence to the maximum length that you specify.
max_length : the maximum length of each sequence. In this example we use 10, but for our actual dataset we will use 512, which is the maximum length of a sequence allowed for BERT.
truncation : if True, then the tokens in each sequence that exceed the maximum length will be truncated.
return_tensors : the type of tensors that will be returned. Since we’re using Pytorch, then we use pt. If you use Tensorflow, then you need to use tf .
'''

from transformers import BertTokenizer

# call the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = 'I will watch Memento tonight'
bert_input = tokenizer(example_text,padding='max_length', max_length = 10,
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tensor([[  101,   146,  1209,  2824,  2508, 26173,  3568,   102,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [18]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] I will watch Memento tonight [SEP] [PAD] [PAD]


## For our case

### Convert the dataset to desired structure:

In [19]:
X_all

array(['i didnt feel humiliated',
       'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
       'im grabbing a minute to post i feel greedy wrong', ...,
       'i feel that i am useful to my people and that gives me a great feeling of achievement',
       'im feeling more comfortable with derby i feel as though i can start to step out my shell',
       'i feel all weird when i have to meet w people i text but like dont talk face to face w'],
      dtype='<U300')

In [20]:
y_all

array([0, 0, 3, ..., 1, 1, 4])

In [21]:
data = pd.DataFrame({
    'category': y_all,
    'text': X_all
})
data.head()

,category,text
0,0,i didnt feel humiliated
1,0,i can go from feeling so hopeless to so damned...
2,3,im grabbing a minute to post i feel greedy wrong
3,2,i am ever feeling nostalgic about the fireplac...
4,3,i am feeling grouchy


In [22]:
category_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}

# Convert numeric values to string representations
data['category'] = data['category'].replace(category_mapping)
data.head()

,category,text
0,sadness,i didnt feel humiliated
1,sadness,i can go from feeling so hopeless to so damned...
2,anger,im grabbing a minute to post i feel greedy wrong
3,love,i am ever feeling nostalgic about the fireplac...
4,anger,i am feeling grouchy


In [23]:
# initial the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

labels = {'sadness':0,
          'joy':1,
          'love':2,
          'anger':3,
          'fear':4,
          'surprise':5
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [24]:
np.random.seed(112)
df_train, df_val, df_test = np.split(data.sample(frac=1, random_state=42),
                                     [int(.8*len(data)), int(.9*len(data))])

print(len(df_train),len(df_val), len(df_test))

16000 2000 2000


In [ ]:
## Train the xiba stuff

In [27]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 6)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [31]:
import os
from google.colab import drive
drive.mount('/content/drive')
datadir = "."
save_dir = "/content/drive/My Drive/model"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
def save_checkpoint(save_dir, model, save_name = 'best_model.pth'):
    save_path = os.path.join(save_dir, save_name)
    torch.save(model.state_dict(), save_path)

def load_model(model, save_dir, save_name = 'best_model.pth'):
    save_path = os.path.join(save_dir, save_name)
    model.load_state_dict(torch.load(save_path))
    return model

In [37]:
#from torchvision import models  # You can replace this with your actual model import
#model_try = models.resnet18(pretrained=True)
#save_checkpoint(save_dir, model_try)
#it works!

In [38]:
#save_checkpoint(save_dir, model)
#loaded_model = load_model(model, save_dir)

In [40]:
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import f1_score

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=4)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    f1_val_list = []

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0
        all_true_train = []
        all_pred_train = []

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()

            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            # Collect true and predicted labels for F1 score calculation
            all_true_train.extend(train_label.cpu().numpy())
            all_pred_train.extend(output.argmax(dim=1).cpu().numpy())

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        total_acc_val = 0
        total_loss_val = 0
        all_true_val = []
        all_pred_val = []

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()

                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc

                # Collect true and predicted labels for F1 score calculation
                all_true_val.extend(val_label.cpu().numpy())
                all_pred_val.extend(output.argmax(dim=1).cpu().numpy())

        # Calculate F1 score for both training and validation sets
        f1_train = f1_score(all_true_train, all_pred_train, average='weighted')
        f1_val = f1_score(all_true_val, all_pred_val, average='weighted')

        #f1_val_list.append(f1_val)

        # Checkpoints are used to save the model with best validation accuracy
        if f1_val >= max(f1_val_list):
          print("Saving Model")
          save_checkpoint(save_dir, model, save_name = 'best_model.pth') # Save model with best performance

        f1_val_list.append(f1_val)

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data):.3f} \
            | Train Accuracy: {total_acc_train / len(train_data):.3f} | Train F1 Score: {f1_train:.3f} \
            | Val Loss: {total_loss_val / len(val_data):.3f} | Val Accuracy: {total_acc_val / len(val_data):.3f} \
            | Val F1 Score: {f1_val:.3f}')

# Example usage
EPOCHS = 20
model = BertClassifier()
LR = 5e-7

train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 4000/4000 [06:30<00:00, 10.25it/s]


Saving Model
Epochs: 1 | Train Loss: 0.381             | Train Accuracy: 0.420 | Train F1 Score: 0.356             | Val Loss: 0.313 | Val Accuracy: 0.576             | Val F1 Score: 0.492


100%|██████████| 4000/4000 [06:30<00:00, 10.25it/s]


Saving Model
Epochs: 2 | Train Loss: 0.271             | Train Accuracy: 0.631 | Train F1 Score: 0.560             | Val Loss: 0.215 | Val Accuracy: 0.733             | Val F1 Score: 0.690


100%|██████████| 4000/4000 [06:32<00:00, 10.18it/s]


...

Epochs: 18 | Train Loss: 0.008             | Train Accuracy: 0.991 | Train F1 Score: 0.991             | Val Loss: 0.067 | Val Accuracy: 0.929             | Val F1 Score: 0.929


100%|██████████| 4000/4000 [06:34<00:00, 10.15it/s]


Epochs: 19 | Train Loss: 0.007             | Train Accuracy: 0.993 | Train F1 Score: 0.993             | Val Loss: 0.069 | Val Accuracy: 0.926             | Val F1 Score: 0.926


100%|██████████| 4000/4000 [06:33<00:00, 10.16it/s]


Epochs: 20 | Train Loss: 0.007             | Train Accuracy: 0.992 | Train F1 Score: 0.992             | Val Loss: 0.071 | Val Accuracy: 0.926             | Val F1 Score: 0.925


In [41]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

def evaluate(model, test_data):
    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for test_input, test_label in test_dataloader:
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            preds = output.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(test_label.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f'Test Accuracy: {accuracy:.3f}')
    print(f'Test F1 Score: {f1:.3f}')

    # Additional information with the full classification report
    report = classification_report(all_labels, all_preds)
    print("Classification Report:\n", report)

# Assuming you have a class Dataset defined and df_test is the test dataset
evaluate(model, df_test)

Test Accuracy: 0.907
Test F1 Score: 0.906
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.95      0.94       569
           1       0.91      0.93      0.92       674
           2       0.83      0.72      0.78       160
           3       0.90      0.93      0.91       272
           4       0.92      0.88      0.90       256
           5       0.81      0.74      0.77        69

    accuracy                           0.91      2000
   macro avg       0.88      0.86      0.87      2000
weighted avg       0.91      0.91      0.91      2000

